## Подготовка

In [17]:
# импортируем нужные библиотеки
import pandas as pd
import numpy as np

from ast import literal_eval

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

from lazypredict.Supervised import LazyClassifier
from xgboost import XGBClassifier 
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from catboost import CatBoostClassifier

In [2]:
# версия решения
version = 20

In [3]:
# пишем класс, вычисляющий групповые статистики
class MeanGrouper():
    """
    Класс, вычисляющий групповые статистики.
    
    Параметры
    ----------    
    group_cols: list
        Список группирующих столбцов.
    agg_col: str
        Агрегируемый столбец.
    agg_func: str
        Агрегирующая функция.
   
    Возвращает
    -------
    X : pandas.DataFrame
        Датафрейм с групповыми статистиками.
    """
    def __init__(self, group_cols, agg_col, agg_func='mean'):
        
        if agg_func not in ['mean', 'median', 'min', 'max', 'std', 'skew']:
            raise ValueError(
                f"Неизвестная агрегирующая функция {agg_func}")
        
        if type(agg_func) != str:
            raise ValueError("Агрегирующая функция должна" + 
                             "иметь строковое значение")
            
        # превращаем столбец в список
        if type(group_cols) != list:
            group_cols = [group_cols]
            
        self.group_cols = group_cols
        self.agg_col = agg_col
        self.agg_func = agg_func
    
    def fit(self, X, y=None):
        
        # проверка наличия пропусков в группирующих столбцах
        if pd.isnull(X[self.group_cols]).any(axis=None) == True:
             raise ValueError(f"Есть пропуски в группирующих столбцах")
        
        # задаем имя
        self.name = '{0}_by_{1}_{2}'.format(
            self.agg_col, self.group_cols, self.agg_func)
        # получаем средние по каждой комбинации категорий
        # признаков в списке
        self.grp = X.groupby(self.group_cols)[[self.agg_col]].agg(
            self.agg_func)
        
        return self 
    
    def transform(self, X, y=None):
        
        # присваиваем имя
        self.grp.columns = [self.name]
        # записываем результаты  
        X = pd.merge(X, self.grp, 
                     left_on=self.group_cols, 
                     right_index=True, how='left')
          
        # пропуски кодируем отдельным значением -1
        X = X.fillna(-1)
        
        return X

In [4]:
# парсер координат
def coord_parser(row):
    if row['type'] == 'Polygon':
        return row['coordinates'][0][0]
    if row['type'] == 'MultiPolygon':
        return row['coordinates'][0][0][0]
    return row['coordinates'][0]['coordinates'][0]

In [5]:
# загрузка тренировочного датасета
df = pd.read_csv('../../data/train_dataset_train.csv')

In [6]:
# функция препроцессинга данных
def preprocessing(df, new_df=False):
    lst = df.filter(regex='nd_mean').columns.tolist()
    lst2 = df.filter(regex='nd_mean').columns.str.lstrip('nd_mean').tolist()
    dictionary = dict(zip(lst, lst2))
    df.rename(columns=dictionary, inplace=True)
    df[['type', 'coordinates']] = pd.DataFrame(
        df['.geo']
        .apply(lambda x: literal_eval(x).values())
        .tolist()
        )
    df[['coord1', 'coord2']] = pd.DataFrame(
        df
        .apply(coord_parser, axis=1)
        .tolist()
        )
    if new_df:
        ident = df['id']       
        X = df.drop(['id', '.geo', 'type', 'coordinates'], axis=1)
    else:
        X = df.drop(['id', '.geo', 'crop', 'type', 'coordinates'], axis=1)
        y = df['crop']
    X = X.sort_index(axis=1)

    X['mean'] = X[X.filter(regex='2021').columns].mean(axis=1)
    X['std'] = X[X.filter(regex='2021').columns].std(axis=1)
    X['diff'] = (X[X.filter(regex='2021').columns].max(axis=1) -
                 X[X.filter(regex='2021').columns].min(axis=1))

    if new_df:
        return X, ident
    else:
        return X, y

In [7]:
# препроцессинг тренировочного датасета
X, y = preprocessing(df)

In [8]:
# загружаем регионы РФ согласно координатам наблюдений
df_regions = pd.read_csv('../../data/regions_hist.csv')

In [9]:
# присоединяем регионы к датасету
X = pd.concat([X, df_regions], axis=1)

## Выбор моделей

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=42,
                                                    stratify=y)

In [19]:
# выберем модели для ансамблирования
lc = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
lazy_models, predictions = lc.fit(X_train, X_test, y_train, y_test)
print(lazy_models)

100%|██████████| 29/29 [00:19<00:00,  1.48it/s]

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
XGBClassifier                      0.96               0.96    None      0.96   
LGBMClassifier                     0.96               0.96    None      0.96   
ExtraTreesClassifier               0.96               0.96    None      0.96   
RandomForestClassifier             0.95               0.95    None      0.95   
SVC                                0.95               0.95    None      0.95   
LinearSVC                          0.94               0.95    None      0.95   
CalibratedClassifierCV             0.94               0.94    None      0.94   
QuadraticDiscriminantAnalysis      0.94               0.94    None      0.94   
LogisticRegression                 0.94               0.94    None      0.94   
RidgeClassifier                    0.94               0.94    None      0.94   
RidgeClassifierCV                  0.94 

## Обучение

In [10]:
# генератор кортежей групповых статистик для пайплайна
list_of_mean_grouper = []
names = []
agg_funcs = ['mean', 'median', 'min', 'max', 'std', 'skew']
agg_cols = X.filter(regex='coord1').columns
group_cols = ['state']
for g in group_cols:
    for ac in agg_cols:
        for f in agg_funcs:
            a = MeanGrouper(
                g, 
                agg_col=ac, 
                agg_func=f)
            name = '+'.join([g,ac,f])
            names.append(name)
            result = (name, a)
            list_of_mean_grouper.append(result)

In [11]:
# лучшие параметры для случайного леса
best_params_rf = {
    "n_estimators": 772,
    "max_depth": 180,
    "min_samples_split": 4,
    "min_samples_leaf": 2,
}

In [12]:
# порядковое кодирование регионов
cat_pipe = Pipeline(
    [
        (
            'cat_encoder',
            OrdinalEncoder(
                handle_unknown='use_encoded_value',
                unknown_value=-1))
        ]
    )
cat_preprocessor = ColumnTransformer(
   [
    ('cat', cat_pipe, ['state'])
    ], remainder='passthrough'
   )

In [13]:
# формирование датасета для финального решения
new = pd.read_csv('../../data/test_dataset_test.csv')
X_new, ident = preprocessing(new, new_df=True)
df_regions_new = pd.read_csv('../../data/regions_new.csv')
X_new = pd.concat([X_new, df_regions_new], axis=1)

In [14]:
# модели для ансамбля
models = [
    CatBoostClassifier,
    RandomForestClassifier,
    XGBClassifier,
    LGBMClassifier,
    ExtraTreesClassifier
        ]

In [15]:
# генерируем экземпляры классов моделей обучения
model_instances = []
random_states = range(5, 30, 1)
for model in models:
    for r in random_states:
        model_instances.append(model().set_params(**{'random_state': r}))

In [16]:
# обучаем модели и ансамблируем решения
probas = []
for model in model_instances:
    if (
        (type(model) != 'lightgbm.sklearn.LGBMClassifier') or
        (type(model) != 'sklearn.ensemble._forest.RandomForestClassifier')):
        grouper = list_of_mean_grouper
    else:
        grouper = []

    clf = Pipeline(
        grouper +
        [
            ('cat_encoder', cat_preprocessor),
            ('scaler', StandardScaler()),
            ('rf', model)
            ])  

    print(clf.named_steps['rf'])  
    clf.fit(X, y)
    proba = clf.predict_proba(X_new)
    probas.append(proba)
    
mean_probas = np.mean(probas, axis=0)
pred = np.argmax(mean_probas, axis=1)

# формируем посылку
pd.DataFrame({'id': ident, 'crop': pred}).to_csv(
    f'../../data/submissions/subm_a{version}.csv', index=False)

Learning rate set to 0.085744
0:	learn: 1.7199947	total: 332ms	remaining: 5m 31s
1:	learn: 1.5532884	total: 440ms	remaining: 3m 39s
2:	learn: 1.4173378	total: 538ms	remaining: 2m 58s
3:	learn: 1.3094447	total: 639ms	remaining: 2m 39s
4:	learn: 1.2254909	total: 736ms	remaining: 2m 26s
5:	learn: 1.1521629	total: 860ms	remaining: 2m 22s
6:	learn: 1.0769754	total: 975ms	remaining: 2m 18s
7:	learn: 1.0167415	total: 1.13s	remaining: 2m 19s
8:	learn: 0.9649809	total: 1.25s	remaining: 2m 17s
9:	learn: 0.9154959	total: 1.37s	remaining: 2m 15s
10:	learn: 0.8729837	total: 1.52s	remaining: 2m 16s
11:	learn: 0.8282794	total: 1.67s	remaining: 2m 17s
12:	learn: 0.7912463	total: 1.85s	remaining: 2m 20s
13:	learn: 0.7524827	total: 1.98s	remaining: 2m 19s
14:	learn: 0.7214053	total: 2.12s	remaining: 2m 18s
15:	learn: 0.6909199	total: 2.24s	remaining: 2m 17s
16:	learn: 0.6654293	total: 2.34s	remaining: 2m 15s
17:	learn: 0.6411250	total: 2.42s	remaining: 2m 12s
18:	learn: 0.6196447	total: 2.52s	remaining: